# Regras do Algoritmo

1. Começamos com 6 pais, nossa geração zero, criados aleatoriamente. Para facilitarmos o código e a lógica do problema, tanto os pais quanto gerações subsequentes terão 1 rainha por linha, para não nos preocuparmos com rainhas em linhas iguais.
2. O gene de cada pai é formado por 8 inteiros entre 1 e 8 (inclusive). O primeiro valor é a coluna que se encontra a rainha da primeira linha, o segundo é a coluna que se encontra a rainha da segunda linha e assim sucessivamente.
3. Recombinamos os pais entre si, escolhendo de maneira aleatória que gene será herdado (do pai A ou B). Como temos muitas combinações possíveis, vamos nos limitar a 2 cruzamento por par de pais, gerando então 30 filhos.
4. Inserimos uma mutação (também aleatória!) em 6 filhos escolhidos aleatoriamente. A mutação consiste na substituição de um valor do gene por outro.
5. Contamos quantos ataques ocorrem por configuração e selecionamos as 5 melhores (menor número de ataques, menor não-aptidão) e um aleatório, descartando o resto.
6. Repetimos o processo a cada nova geração, até termos 5 gerações.

In [1]:
from random import randint
from collections import Counter

In [2]:
def geracao_inicial():
    """Retorna uma lista com 6 pais para a geracao 0, cada um com 8 inteiros em [1, 8]."""
    return [[randint(1, 8) for y in range(8)] for x in range(6)]

In [3]:
def recombinacao(pais):
    """Retorna uma lista com filhos gerados de recombinacoes aleatorias."""
    
    filhos = []
    for x in range(6):
        for y in range(6):
            if (x != y):
                filhos += [[(pais[x][k] if (randint(0, 1) == 0) else pais[y][k]) for k in range(8)]]
    return filhos

In [4]:
def mutacao(filhos):
    """Insere um gene de mutacao aleatorio em 6 filhos aleatorios."""
    
    mutantes = []
    index = []
    
    # Para que nao repetamos filhos, esse bloco gera 6 indices diferentes.
    while (len(index) != 6):
        p_i = randint(0, 29)
        if not (p_i in index):
            index += [p_i]
    
    # Inserimos a mutacao em cada filho.
    for i in index:
        mut = filhos[i]
        i_m = randint(0, 7)
        novo_gene = randint(1, 8)
        while (mut[i_m] == novo_gene): # Se o novo valor for o mesmo que o antigo,
            novo_gene = randint(1, 8)  # nao havera mutacao! Precisamos checar isso
        mut[i_m] = novo_gene
        mutantes += [mut]
        
    return mutantes

In [5]:
def contagem_ataques(gene_pool):
    """Retorna a quantidade de ataques presentes em cada membro de gene_pool."""
    
    # Para uma das checagens diagonais, precisamos de um vetor de pesos.
    pesos = [7, 5, 3, 1, -1, -3, -5, -7]
    ataques = []
    
    for ser in gene_pool:
        diagonais_1 = [x + ser[x] for x in range(8)]
        diagonais_2 = [x + ser[x] + pesos[x] for x in range(8)]
        qtd = 0
        for l in range(8):
            qtd += Counter(ser)[ser[l]] - 1
            qtd += Counter(diagonais_1)[diagonais_1[l]] - 1
            qtd += Counter(diagonais_2)[diagonais_2[l]] - 1
        ataques += [int(qtd/2)]
    
    return ataques

In [6]:
def melhores_pior(gene_pool, ataques):
    """Escolhe os 5 melhores e um piot ser de gene_pool, baseado em suas qtds de ataques."""
    
    # Juntamos os dois vetores e os organizamos pela quantidade de ataques.
    ser_ataque = [[gene_pool[k], ataques[k]] for k in range(len(gene_pool))]
    organizado = sorted(ser_ataque, key=lambda e: e[1])
    melhor_pior = organizado[:5] + [organizado[randint(5, len(organizado)-1)]]
    
    # Imprimimos na tela os escolhidos.
    for x in melhor_pior:
        print("Gene: {} // Ataques: {}".format(x[0], x[1]))
        
    return [x[0] for x in melhor_pior]

## Hora de testar o código!

### Geração 1

In [7]:
pais1 = geracao_inicial()
pais1

[[4, 4, 5, 1, 5, 4, 1, 3],
 [4, 1, 1, 5, 6, 3, 6, 8],
 [2, 4, 2, 1, 4, 6, 8, 2],
 [3, 7, 2, 5, 5, 6, 7, 6],
 [1, 6, 4, 7, 4, 3, 8, 6],
 [8, 2, 7, 2, 4, 1, 1, 4]]

In [8]:
atk_pais1 = contagem_ataques(pais1)
print(atk_pais1)

[9, 5, 7, 8, 4, 4]


In [9]:
fils1 = recombinacao(pais1)
fils1

[[4, 4, 5, 5, 6, 4, 1, 3],
 [4, 4, 2, 1, 4, 4, 1, 3],
 [3, 4, 5, 1, 5, 6, 7, 3],
 [4, 6, 4, 1, 5, 3, 1, 3],
 [4, 4, 7, 2, 4, 1, 1, 4],
 [4, 1, 5, 5, 5, 4, 6, 8],
 [4, 4, 2, 1, 4, 3, 6, 8],
 [3, 7, 1, 5, 6, 3, 7, 6],
 [1, 6, 4, 7, 6, 3, 6, 8],
 [4, 1, 7, 2, 4, 3, 6, 4],
 [2, 4, 2, 1, 5, 4, 8, 2],
 [4, 4, 2, 1, 4, 6, 8, 8],
 [2, 4, 2, 1, 4, 6, 7, 2],
 [1, 6, 4, 7, 4, 3, 8, 2],
 [2, 4, 7, 1, 4, 1, 8, 2],
 [4, 7, 5, 5, 5, 4, 1, 3],
 [4, 7, 2, 5, 6, 3, 6, 6],
 [3, 7, 2, 5, 4, 6, 7, 6],
 [3, 6, 4, 5, 4, 6, 7, 6],
 [3, 7, 2, 5, 5, 1, 1, 6],
 [1, 4, 4, 1, 4, 3, 1, 6],
 [4, 1, 4, 5, 4, 3, 8, 8],
 [2, 6, 2, 7, 4, 6, 8, 2],
 [1, 6, 4, 5, 4, 3, 7, 6],
 [8, 2, 4, 2, 4, 3, 1, 4],
 [4, 2, 5, 2, 4, 4, 1, 3],
 [4, 1, 1, 2, 6, 1, 1, 4],
 [2, 2, 2, 2, 4, 6, 8, 4],
 [3, 2, 7, 2, 4, 1, 7, 4],
 [1, 2, 7, 2, 4, 3, 8, 4]]

In [10]:
atk_fils1 = contagem_ataques(fils1)
print(atk_fils1)

[8, 11, 8, 6, 8, 7, 11, 10, 6, 7, 9, 9, 7, 3, 5, 7, 9, 8, 8, 3, 8, 11, 6, 8, 7, 6, 8, 11, 5, 4]


In [11]:
muts1 = mutacao(fils1)
muts1

[[8, 2, 4, 2, 4, 3, 3, 4],
 [4, 7, 5, 5, 5, 4, 4, 3],
 [6, 6, 4, 7, 4, 3, 8, 2],
 [5, 4, 7, 1, 4, 1, 8, 2],
 [4, 1, 5, 5, 5, 4, 6, 6],
 [2, 2, 3, 2, 4, 6, 8, 4]]

In [12]:
atk_muts1 = contagem_ataques(muts1)
print(atk_muts1)

[9, 9, 5, 5, 8, 10]


In [13]:
gp1 = pais1 + fils1 + muts1
atk_gp1 = atk_pais1 + atk_fils1 + atk_muts1

In [14]:
pais2 = melhores_pior(gp1, atk_gp1)

Gene: [6, 6, 4, 7, 4, 3, 8, 2] // Ataques: 3
Gene: [3, 7, 2, 5, 5, 1, 1, 6] // Ataques: 3
Gene: [1, 6, 4, 7, 4, 3, 8, 6] // Ataques: 4
Gene: [8, 2, 7, 2, 4, 1, 1, 4] // Ataques: 4
Gene: [1, 2, 7, 2, 4, 3, 8, 4] // Ataques: 4
Gene: [2, 2, 3, 2, 4, 6, 8, 4] // Ataques: 11


### Geração 2

In [15]:
atk_pais2 = contagem_ataques(pais2)
print(atk_pais2)

[5, 3, 4, 4, 4, 10]


In [16]:
fils2 = recombinacao(pais2)
fils2

[[3, 6, 4, 5, 4, 1, 1, 2],
 [6, 6, 4, 7, 4, 3, 8, 2],
 [6, 2, 4, 7, 4, 3, 1, 4],
 [1, 6, 7, 2, 4, 3, 8, 4],
 [6, 2, 4, 2, 4, 3, 8, 2],
 [6, 7, 2, 7, 4, 1, 1, 2],
 [1, 7, 2, 7, 4, 1, 1, 6],
 [3, 7, 2, 2, 4, 1, 1, 4],
 [1, 2, 7, 2, 4, 1, 1, 6],
 [3, 7, 3, 2, 5, 1, 8, 6],
 [6, 6, 4, 7, 4, 3, 8, 2],
 [3, 6, 2, 5, 5, 3, 1, 6],
 [8, 2, 7, 2, 4, 1, 1, 4],
 [1, 2, 7, 7, 4, 3, 8, 6],
 [2, 6, 3, 2, 4, 6, 8, 4],
 [8, 6, 7, 2, 4, 1, 1, 2],
 [8, 2, 2, 5, 4, 1, 1, 4],
 [8, 6, 4, 7, 4, 3, 1, 4],
 [1, 2, 7, 2, 4, 3, 8, 4],
 [2, 2, 3, 2, 4, 6, 1, 4],
 [1, 2, 7, 7, 4, 3, 8, 2],
 [3, 7, 7, 2, 4, 1, 8, 6],
 [1, 6, 7, 2, 4, 3, 8, 6],
 [1, 2, 7, 2, 4, 3, 8, 4],
 [2, 2, 3, 2, 4, 6, 8, 4],
 [2, 2, 3, 2, 4, 6, 8, 2],
 [2, 2, 2, 2, 4, 6, 1, 6],
 [2, 2, 4, 2, 4, 6, 8, 6],
 [2, 2, 3, 2, 4, 1, 8, 4],
 [2, 2, 7, 2, 4, 3, 8, 4]]

In [17]:
atk_fils2 = contagem_ataques(fils2)
print(atk_fils2)

[7, 5, 5, 3, 7, 8, 6, 5, 6, 4, 5, 5, 4, 3, 7, 7, 7, 7, 4, 9, 5, 3, 4, 4, 10, 11, 10, 10, 7, 6]


In [18]:
muts2 = mutacao(fils2)
muts2

[[3, 7, 3, 2, 5, 2, 8, 6],
 [2, 2, 2, 2, 4, 3, 8, 4],
 [2, 2, 3, 2, 3, 6, 8, 4],
 [2, 2, 3, 8, 4, 6, 8, 2],
 [1, 6, 6, 2, 4, 3, 8, 4],
 [1, 5, 2, 7, 4, 1, 1, 6]]

In [19]:
atk_muts2 = contagem_ataques(muts2)
print(atk_muts2)

[5, 10, 11, 9, 5, 6]


In [20]:
gp2 = pais2 + fils2 + muts2
atk_gp2 = atk_pais2 + atk_fils2 + atk_muts2

In [21]:
pais3 = melhores_pior(gp2, atk_gp2)

Gene: [3, 7, 2, 5, 5, 1, 1, 6] // Ataques: 3
Gene: [1, 6, 6, 2, 4, 3, 8, 4] // Ataques: 3
Gene: [1, 2, 7, 7, 4, 3, 8, 6] // Ataques: 3
Gene: [3, 7, 7, 2, 4, 1, 8, 6] // Ataques: 3
Gene: [1, 6, 4, 7, 4, 3, 8, 6] // Ataques: 4
Gene: [1, 5, 2, 7, 4, 1, 1, 6] // Ataques: 6


### Geração 3

In [22]:
atk_pais3 = contagem_ataques(pais3)
print(atk_pais3)

[3, 5, 3, 3, 4, 6]


In [23]:
fils3 = recombinacao(pais3)
fils3

[[1, 6, 6, 2, 4, 1, 1, 4],
 [1, 7, 2, 5, 4, 1, 1, 6],
 [3, 7, 7, 5, 5, 1, 8, 6],
 [3, 7, 2, 5, 5, 3, 1, 6],
 [3, 5, 2, 7, 5, 1, 1, 6],
 [3, 7, 6, 2, 4, 1, 1, 4],
 [1, 6, 7, 2, 4, 3, 8, 4],
 [1, 6, 7, 2, 4, 3, 8, 4],
 [1, 6, 6, 2, 4, 3, 8, 4],
 [1, 5, 6, 2, 4, 1, 1, 4],
 [3, 7, 2, 7, 4, 1, 1, 6],
 [1, 6, 7, 2, 4, 3, 8, 6],
 [3, 7, 7, 2, 4, 1, 8, 6],
 [1, 2, 7, 7, 4, 3, 8, 6],
 [1, 2, 7, 7, 4, 1, 8, 6],
 [3, 7, 7, 2, 5, 1, 8, 6],
 [3, 7, 6, 2, 4, 3, 8, 4],
 [1, 7, 7, 2, 4, 3, 8, 6],
 [1, 6, 4, 2, 4, 3, 8, 6],
 [3, 7, 2, 2, 4, 1, 8, 6],
 [3, 7, 2, 7, 5, 3, 8, 6],
 [1, 6, 6, 7, 4, 3, 8, 4],
 [1, 6, 7, 7, 4, 3, 8, 6],
 [1, 7, 4, 2, 4, 1, 8, 6],
 [1, 6, 4, 7, 4, 3, 8, 6],
 [3, 7, 2, 5, 5, 1, 1, 6],
 [1, 6, 6, 7, 4, 3, 8, 6],
 [1, 5, 7, 7, 4, 1, 8, 6],
 [3, 7, 2, 2, 4, 1, 8, 6],
 [1, 6, 4, 7, 4, 3, 8, 6]]

In [24]:
atk_fils3 = contagem_ataques(fils3)
print(atk_fils3)

[7, 7, 5, 5, 4, 5, 3, 3, 5, 7, 4, 4, 3, 3, 3, 3, 8, 5, 5, 4, 3, 6, 4, 6, 4, 3, 7, 4, 4, 4]


In [25]:
muts3 = mutacao(fils3)
muts3

[[1, 6, 7, 2, 4, 1, 8, 4],
 [1, 6, 6, 2, 4, 4, 1, 4],
 [1, 6, 4, 6, 4, 3, 8, 6],
 [8, 6, 6, 2, 4, 3, 8, 4],
 [1, 6, 7, 8, 4, 3, 8, 6],
 [3, 7, 2, 2, 4, 5, 8, 6]]

In [26]:
atk_muts3 = contagem_ataques(muts3)
print(atk_muts3)

[3, 8, 6, 9, 6, 6]


In [27]:
gp3 = pais3 + fils3 + muts3
atk_gp3 = atk_pais3 + atk_fils3 + atk_muts3

In [28]:
pais4 = melhores_pior(gp3, atk_gp3)

Gene: [3, 7, 2, 5, 5, 1, 1, 6] // Ataques: 3
Gene: [1, 2, 7, 7, 4, 3, 8, 6] // Ataques: 3
Gene: [3, 7, 7, 2, 4, 1, 8, 6] // Ataques: 3
Gene: [1, 6, 7, 2, 4, 3, 8, 4] // Ataques: 3
Gene: [1, 6, 7, 2, 4, 1, 8, 4] // Ataques: 3
Gene: [8, 6, 6, 2, 4, 3, 8, 4] // Ataques: 9


### Geração 4

In [29]:
atk_pais4 = contagem_ataques(pais4)
print(atk_pais4)

[3, 3, 3, 3, 3, 9]


In [30]:
fils4 = recombinacao(pais4)
fils4

[[3, 2, 2, 7, 4, 3, 8, 6],
 [3, 7, 2, 2, 4, 1, 1, 6],
 [1, 6, 7, 5, 5, 1, 1, 4],
 [3, 6, 7, 5, 5, 1, 1, 6],
 [8, 7, 2, 5, 5, 3, 1, 4],
 [1, 2, 2, 5, 5, 3, 1, 6],
 [1, 7, 7, 2, 4, 1, 8, 6],
 [1, 6, 7, 2, 4, 3, 8, 4],
 [1, 6, 7, 7, 4, 1, 8, 6],
 [8, 2, 7, 7, 4, 3, 8, 4],
 [3, 7, 2, 2, 5, 1, 1, 6],
 [1, 2, 7, 7, 4, 1, 8, 6],
 [3, 7, 7, 2, 4, 3, 8, 4],
 [1, 6, 7, 2, 4, 1, 8, 4],
 [8, 6, 6, 2, 4, 1, 8, 6],
 [1, 7, 2, 5, 5, 3, 8, 4],
 [1, 2, 7, 7, 4, 3, 8, 4],
 [1, 6, 7, 2, 4, 3, 8, 4],
 [1, 6, 7, 2, 4, 1, 8, 4],
 [1, 6, 7, 2, 4, 3, 8, 4],
 [3, 7, 2, 2, 5, 1, 8, 4],
 [1, 6, 7, 2, 4, 3, 8, 4],
 [1, 7, 7, 2, 4, 1, 8, 4],
 [1, 6, 7, 2, 4, 3, 8, 4],
 [1, 6, 6, 2, 4, 1, 8, 4],
 [3, 7, 2, 2, 5, 1, 1, 4],
 [1, 2, 6, 7, 4, 3, 8, 6],
 [3, 7, 6, 2, 4, 1, 8, 4],
 [8, 6, 6, 2, 4, 3, 8, 4],
 [8, 6, 6, 2, 4, 3, 8, 4]]

In [31]:
atk_fils4 = contagem_ataques(fils4)
print(atk_fils4)

[5, 5, 8, 6, 7, 7, 4, 3, 4, 6, 3, 3, 6, 3, 8, 6, 4, 3, 3, 3, 1, 3, 4, 3, 4, 2, 6, 4, 9, 9]


In [32]:
muts4 = mutacao(fils4)
muts4

[[8, 2, 7, 7, 4, 4, 8, 4],
 [1, 6, 7, 2, 4, 5, 8, 4],
 [3, 7, 2, 7, 4, 1, 1, 6],
 [8, 7, 2, 5, 5, 3, 1, 3],
 [3, 7, 2, 6, 5, 1, 1, 6],
 [8, 6, 6, 2, 4, 3, 2, 4]]

In [33]:
atk_muts4 = contagem_ataques(muts4)
print(atk_muts4)

[7, 3, 4, 8, 4, 13]


In [34]:
gp4 = pais4 + fils4 + muts4
atk_gp4 = atk_pais4 + atk_fils4 + atk_muts4

In [35]:
pais5 = melhores_pior(gp4, atk_gp4)

Gene: [3, 7, 2, 2, 5, 1, 8, 4] // Ataques: 1
Gene: [3, 7, 2, 2, 5, 1, 1, 4] // Ataques: 2
Gene: [3, 7, 2, 5, 5, 1, 1, 6] // Ataques: 3
Gene: [1, 2, 7, 7, 4, 3, 8, 6] // Ataques: 3
Gene: [3, 7, 7, 2, 4, 1, 8, 6] // Ataques: 3
Gene: [1, 6, 7, 2, 4, 1, 8, 4] // Ataques: 3


### Geração 5

In [36]:
atk_pais5 = contagem_ataques(pais5)
print(atk_pais5)

[1, 2, 3, 3, 3, 3]


In [37]:
fils5 = recombinacao(pais5)
fils5

[[3, 7, 2, 2, 5, 1, 8, 4],
 [3, 7, 2, 5, 5, 1, 8, 4],
 [3, 2, 7, 7, 5, 1, 8, 4],
 [3, 7, 2, 2, 4, 1, 8, 6],
 [3, 7, 2, 2, 4, 1, 8, 4],
 [3, 7, 2, 2, 5, 1, 1, 4],
 [3, 7, 2, 2, 5, 1, 1, 6],
 [1, 2, 7, 7, 5, 3, 8, 4],
 [3, 7, 7, 2, 5, 1, 8, 6],
 [3, 7, 2, 2, 5, 1, 8, 4],
 [3, 7, 2, 2, 5, 1, 1, 4],
 [3, 7, 2, 5, 5, 1, 1, 6],
 [1, 7, 2, 7, 4, 3, 8, 6],
 [3, 7, 2, 5, 5, 1, 8, 6],
 [3, 6, 7, 2, 5, 1, 8, 4],
 [1, 7, 7, 7, 4, 1, 8, 4],
 [1, 2, 7, 7, 4, 3, 1, 6],
 [1, 7, 2, 5, 4, 1, 1, 6],
 [1, 7, 7, 7, 4, 1, 8, 6],
 [1, 6, 7, 2, 4, 3, 8, 4],
 [3, 7, 2, 2, 5, 1, 8, 6],
 [3, 7, 2, 2, 4, 1, 8, 4],
 [3, 7, 7, 2, 4, 1, 8, 6],
 [1, 2, 7, 2, 4, 3, 8, 6],
 [1, 6, 7, 2, 4, 1, 8, 6],
 [3, 6, 2, 2, 4, 1, 8, 4],
 [1, 6, 2, 2, 5, 1, 1, 4],
 [1, 7, 2, 5, 5, 1, 8, 4],
 [1, 6, 7, 2, 4, 3, 8, 6],
 [1, 6, 7, 2, 4, 1, 8, 4]]

In [38]:
atk_fils5 = contagem_ataques(fils5)
print(atk_fils5)

[1, 3, 4, 4, 4, 2, 3, 5, 3, 1, 2, 3, 5, 3, 2, 6, 4, 7, 5, 3, 2, 4, 3, 4, 4, 3, 6, 5, 4, 3]


In [39]:
muts5 = mutacao(fils5)
muts5

[[3, 7, 2, 2, 5, 1, 8, 1],
 [1, 7, 8, 7, 4, 1, 8, 6],
 [3, 7, 2, 2, 5, 5, 1, 4],
 [3, 7, 2, 2, 5, 1, 8, 2],
 [6, 2, 7, 7, 4, 3, 1, 6],
 [1, 7, 4, 5, 4, 1, 1, 6]]

In [40]:
atk_muts5 = contagem_ataques(muts5)
print(atk_muts5)

[3, 6, 3, 4, 3, 9]


In [41]:
gp5 = pais5 + fils5 + muts5
atk_gp5 = atk_pais5 + atk_fils5 + atk_muts5

### Geração Final

In [42]:
pais_final = melhores_pior(gp5, atk_gp5)

Gene: [3, 7, 2, 2, 5, 1, 8, 4] // Ataques: 1
Gene: [3, 7, 2, 2, 5, 1, 8, 2] // Ataques: 1
Gene: [3, 7, 2, 2, 5, 1, 8, 1] // Ataques: 1
Gene: [3, 7, 2, 2, 5, 1, 1, 4] // Ataques: 2
Gene: [3, 7, 2, 2, 5, 1, 1, 4] // Ataques: 2
Gene: [3, 7, 2, 2, 5, 5, 1, 4] // Ataques: 3
